## Moveshelf API upload example
This interactive notebook is intended to guide a user to upload data into Moveshelf using the API.

The datastructure of Moveshelf is organized as follows:
* Project: Projects are the highest level and associated to a single organization in Moveshelf. 
* Subjects: Each project contains a list of subjects. At project level, access to the Electronic Health Record (EHR) of a subject can be made. 
* Sessions: A session contains the relevant information for a specific measurement session and is typically defined by the date of the measurement. 
* Conditions: Conditions specify a group of trials that were performed within a session. 
* Trials: Trials, aka clips, are containers used to store our data. It consists of metadata and 'Additional Data', where the actual data of a trial is stored. 

Projects, Subjects and sessions are defined by their ID. When uploading data, a clip id is needed, which can be generated for new trial/clip, or obtained from the existing clips. Using this id, additional data can be uploaded using the provided upload url.

For the data to be uploaded, the type needs to be specified. Within Moveshelf, we support the following data types (to be specified in the upload):
* video: .mp4, .mov, .mpg, .avi
* motion: .bvh, .fbx, .trc, .glb, .mox, c3d, xlsx
* doc: .pdf
* data: .csv, .json, .txt
* img: .png, .jpg
* camera: .xcp
* raw: anything not specified above


### Prerequisites
* Basic Python understanding
* A Moveshelf account with API key (the API key can be obtained from your account settings in Moveshelf. Make sure your API key ('mvshlf-api-key.json') is stored in the root folder of this project, or provide the path during setup of the API. 
* A local copy of Moveshelf Python API example Github repository (https://github.com/moveshelf/python-api-example)

## Setup your environment

### Import dependencies

In [ ]:
import os, sys, json
parentFolder = os.path.dirname(os.getcwd())
sys.path.append(parentFolder)
!pip install -r ../requirements.txt


### Specify the details of your data to be uploaded and where it should go

In [ ]:
dataPath = r'<Path of folder where data is located>'  # e.g. r'C:\Users\testUser\Data\testTrial1'
filesToUpload = ['<list of files to upload>']  # list of files to be uploaded
dataType = '<data_type>'   # type of the data to be uploaded, see above for definition, e.g. 'data'

myProject = '<user>/<projectName>' # e.g. support/demoProject
mySubject = '<name>' # subject name, e.g. Subject1
mySession = '<session_name(typical date)>' # session name, e.g. 2021-01-01
myCondition = '<condition_name'   # condition name, e.g. 2-min walk
myTrial = '<trial_name>'   # trial name, e.g. Trial-1

### Import necessary packages

In [ ]:
import IPython
from api import MoveshelfApi, Metadata
import util

### Setup the API 

In [ ]:
# Load the default configuration
with open(os.path.join(parentFolder,'mvshlf-config.spec.json'), 'r') as configFile:
    data = json.load(configFile)

# And overwrite with personal configuration if available
personalConfig = os.path.join(parentFolder,'mvshlf-config.json')
if os.path.isfile(personalConfig):
    with open(personalConfig, 'r') as configFile:
        data.update(json.load(configFile))

api = MoveshelfApi(api_key_file = os.path.join(parentFolder,data['apiKeyFileName']), api_url = data['apiUrl'])

### Get available projects

In [ ]:
projects = api.getUserProjects()
projectNames = [project['name'] for project in projects if len(projects) > 0]
print('Available projects:')
print(*projectNames, sep='\n')


### Select the project

In [ ]:
projectNames = [p['name'] for p in projects]
iMyProject = projectNames.index(myProject)
myProjectId = projects[iMyProject]['id']
print('Project ID is: ' + myProjectId)


### Find the subject

In [ ]:
subjects = api.getProjectSubjects(myProjectId)
subjectNames = [s['name'] for s in subjects]

if mySubject not in subjectNames:
    # create Subject
    subject = api.createSubject(myProject, mySubject)
    mySubjectId = subject['id']
else:
    # get subject data
    iMySubject = subjectNames.index(mySubject)
    mySubjectId = subjects[iMySubject]['id']
       
# Extract subject details
subjectDetails = api.getSubjectDetails(mySubjectId)
subjectName = subjectDetails['name']

print('Subject found, name is: ' + subjectName + ', subject ID is: ' + mySubjectId)

### Get session or create new

In [ ]:
sessions = subjectDetails['sessions']
sessionExists = False
for session in sessions:
    try:
        sessionName = session['projectPath'].split('/')[2]
    except:
        sessionName = ""
    if sessionName == mySession:
        sessionId = session['id']
        sessionExists = True
        print('Session found')
        break

if not sessionExists:
    sessionPath = '/' + subjectName + '/' + mySession + '/'
    session = api.createSession(myProject, sessionPath, mySubjectId)
    sessionId = session['id']
    print('Session created')

session = api.getSessionById(sessionId)

print('Session name is: ' + sessionName + ', session ID is: ' + sessionId)

### Get condition name or add new

In [ ]:
conditions = []
conditions = util.getConditionsFromSession(session, conditions)

condition = {}
for c in conditions:
    if (c['path'] == myCondition):
        condition = c
        break

if (not condition):
    condition['path'] = myCondition
    condition['clips'] = []


### Get clip id

In [ ]:
clipId = util.addOrGetTrial(api, session, condition, myTrial)
print('Clip id is: ' + clipId) 

In [ ]:
existingAdditionalData = api.getAdditionalData(clipId)
existingFileNames = [data['originalFileName'] for data in existingAdditionalData if len(existingAdditionalData) > 0]

print('Existing data for clip: ')
print(*existingFileNames, sep = "\n")

### Upload data


In [ ]:
for fileName in filesToUpload:
    filePath = os.path.join(dataPath, fileName)
    
    if fileName in existingFileNames:
        print(fileName + ' was found in clip, will skip this data.')
        continue

    print('Uploading data for : ' + myCondition + ', ' + myTrial + ': ' + fileName)
    
    dataId = api.uploadAdditionalData(filePath, clipId, dataType, fileName)